# Fake or Real: Classical ML Baselines for Deepfake Audio Detection
**By Faheem Ahmad**

Local version for Mac M4 (Apple Silicon GPU via MPS) — equivalent to the Colab notebook.

In [1]:
# =========================================
# 0) Imports & Setup (Local Mac M4 GPU)
# =========================================
import os, glob, warnings, hashlib
import numpy as np
import pandas as pd
import librosa
import librosa.feature as LF
import soundfile as sf
from scipy import signal, stats
import matplotlib
matplotlib.use('Agg')  # non-interactive backend for local run
import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel, delayed
from itertools import combinations
from scipy.stats import chi2

from sklearn.model_selection import (
    GroupKFold, GroupShuffleSplit, StratifiedKFold,
    GridSearchCV, train_test_split
)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, roc_auc_score, roc_curve, auc,
    confusion_matrix, classification_report, det_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
)
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# GPU: Apple M4 via PyTorch MPS + torchaudio for GPU-accelerated audio
import torch
import torchaudio
import torchaudio.transforms as T

DEVICE = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f'PyTorch device: {DEVICE}')
print(f'Apple M4 GPU available: {torch.backends.mps.is_available()}')
print(f'PyTorch version: {torch.__version__}')
print(f'torchaudio version: {torchaudio.__version__}')

# Quick GPU smoke test
_t = torch.randn(256, 256, device=DEVICE)
_ = torch.mm(_t, _t)
print(f'GPU smoke test passed (256x256 matmul on {DEVICE})')

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 130

SEED = 4
np.random.seed(SEED)
torch.manual_seed(SEED)

print('All imports successful!')

PyTorch device: mps
Apple M4 GPU available: True
PyTorch version: 2.10.0
torchaudio version: 2.10.0
GPU smoke test passed (256x256 matmul on mps)
All imports successful!


In [2]:
# =========================================
# 1) Configuration & Dataset Paths (Local)
# =========================================

# LOCAL path to the FoR dataset (unzipped from FoR_root.zip)
BASE_DIR = os.path.expanduser('~/Downloads/FoR_root')

# Subpaths to search
SUBSETS = [
    'for-2sec/for-2seconds/training/fake',
    'for-2sec/for-2seconds/training/real',
    'for-2sec/for-2seconds/testing/fake',
    'for-2sec/for-2seconds/testing/real',
    'for-2sec/for-2seconds/validation/fake',
    'for-2sec/for-2seconds/validation/real',
    'for-rerec/for-rerecorded/training/fake',
    'for-rerec/for-rerecorded/training/real',
    'for-rerec/for-rerecorded/testing/fake',
    'for-rerec/for-rerecorded/testing/real',
    'for-rerec/for-rerecorded/validation/fake',
    'for-rerec/for-rerecorded/validation/real',
]

# Verify dataset exists
assert os.path.isdir(BASE_DIR), f'Dataset not found at {BASE_DIR}. Unzip FoR_root.zip first.'

for rel in SUBSETS:
    full_path = os.path.join(BASE_DIR, rel)
    exists = os.path.isdir(full_path)
    print(f"{'OK' if exists else 'MISSING'}: {full_path}")

OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/training/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/training/real
OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/testing/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/testing/real
OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/validation/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-2sec/for-2seconds/validation/real
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/training/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/training/real
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/testing/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/testing/real
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/validation/fake
OK: /Users/faheemahmad/Downloads/FoR_root/for-rerec/for-rerecorded/validation/real


In [3]:
# =========================================
# 2) Auto-Repair: Normalize audio to 16kHz
#    mono PCM WAV, remove broken/silent files
# =========================================

# Thresholds
MIN_FILESIZE_BYTES = 8_000
MIN_DURATION_SEC   = 0.20
TARGET_SR          = 16000

def _looks_suspicious(fname):
    low = fname.lower()
    bad_suffixes = [
        '.mp3.wav', '.m4a.wav', '.aac.wav', '.ogg.wav', '.wma.wav',
        '.wav_mono.wav', '.wav_norm.wav', '.wav_silence.wav', '_2sec.wav'
    ]
    return (any(low.endswith(suf) for suf in bad_suffixes) or
            low.endswith(('.mp3', '.m4a', '.aac', '.ogg', '.wma', '.flac')))

def _safe_decode(path, sr=TARGET_SR):
    try:
        y, srr = librosa.load(path, sr=sr, mono=True)
        return y, sr
    except Exception:
        return None, None

def _reencode_to_wav(src_path):
    y, sr = _safe_decode(src_path, sr=TARGET_SR)
    if y is None or len(y) == 0:
        return None
    base, _ = os.path.splitext(src_path)
    for suf in ['.mp3', '.m4a', '.aac', '.ogg', '.wma', '.flac',
                '.wav_mono', '.wav_norm', '.wav_silence', '_2sec']:
        if base.lower().endswith(suf):
            base = base[:-len(suf)]
    out_path = base + '.wav'
    sf.write(out_path, y, TARGET_SR, subtype='PCM_16')
    if os.path.abspath(out_path) != os.path.abspath(src_path) and os.path.exists(src_path):
        try: os.remove(src_path)
        except: pass
    return out_path

def _is_silent(y, sr):
    if y is None or len(y) == 0:
        return True
    if len(y) / sr < MIN_DURATION_SEC:
        return True
    rms = np.sqrt(np.mean(y**2)) if len(y) else 0.0
    return bool(rms < 1e-4)

repaired, dropped = 0, 0
all_candidates = glob.glob(os.path.join(BASE_DIR, '**/*.*'), recursive=True)

for p in all_candidates:
    if not os.path.isfile(p):
        continue
    if not p.lower().endswith(('.wav', '.mp3', '.m4a', '.aac', '.ogg', '.wma', '.flac')):
        continue
    try:
        if os.path.getsize(p) < MIN_FILESIZE_BYTES:
            os.remove(p)
            dropped += 1
            continue
    except: pass

    if _looks_suspicious(p) or not p.lower().endswith('.wav'):
        out = _reencode_to_wav(p)
        if out is None:
            try: os.remove(p)
            except: pass
            dropped += 1
            continue
        else:
            repaired += 1
            p = out

    y, sr = _safe_decode(p, sr=TARGET_SR)
    if y is None or _is_silent(y, sr):
        try: os.remove(p)
        except: pass
        dropped += 1

print(f'Repair summary -> Repaired: {repaired} | Dropped: {dropped}')

Repair summary -> Repaired: 0 | Dropped: 0


In [4]:
# =========================================
# 2b) Discover paths, labels, groups
# =========================================

def derive_group_from_filename(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    if '_' in stem:
        return stem.split('_')[0]
    h = hashlib.md5(stem.encode('utf-8')).hexdigest()[:8]
    return f'spk{h}'

def discover_paths_labels_groups(base_dir):
    X_paths, y, groups = [], [], []
    for rel in SUBSETS:
        full = os.path.join(base_dir, rel)
        label = 1 if full.lower().endswith('fake') else 0
        subset_tag = 'for-2seconds' if 'for-2seconds' in full.lower() else 'for-rerecorded'
        if not os.path.isdir(full):
            continue
        for fname in os.listdir(full):
            fpath = os.path.join(full, fname)
            if os.path.isfile(fpath) and fname.lower().endswith('.wav'):
                X_paths.append(fpath)
                y.append(label)
                groups.append(f'{subset_tag}:{derive_group_from_filename(fpath)}')
    if not X_paths:
        raise RuntimeError('No usable WAV files. Check BASE_DIR and folder structure.')
    return X_paths, np.array(y, dtype=int), np.array(groups)

paths, labels, groups = discover_paths_labels_groups(BASE_DIR)
print(f'Usable files: {len(paths)} | Class counts [real(0), fake(1)]: {np.bincount(labels)}')

Usable files: 31138 | Class counts [real(0), fake(1)]: [15548 15590]


In [5]:
# =========================================
# 3) Feature Extraction at 44.1kHz AND 16kHz
#    GPU-ACCELERATED: torchaudio spectral features on MPS
#    CPU parallel: librosa prosody features (F0/YIN)
# =========================================

import time

# Shared constants
TARGET_SR_16K = 16000
HOP_16K       = 160       # ~10 ms at 16 kHz
FRAME_16K     = 1024

SR_44100      = 44100
FRAME_44K     = int(round(0.025 * SR_44100))   # 25 ms window
HOP_44K       = int(round(0.010 * SR_44100))   # 10 ms hop

F0_FMIN       = 50
F0_FMAX       = 500

# ---- Stats helper ----
def _safe_stats(x):
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return dict(mean=np.nan, std=np.nan, p10=np.nan, p90=np.nan,
                    iqr=np.nan, rng=np.nan, cv=np.nan)
    q10, q90 = np.quantile(x, [0.10, 0.90])
    iqr = np.quantile(x, 0.75) - np.quantile(x, 0.25)
    rng = x.max() - x.min()
    std = x.std()
    mean = x.mean()
    cv = std / mean if mean else np.nan
    return dict(mean=mean, std=std, p10=q10, p90=q90, iqr=iqr, rng=rng, cv=cv)

# ---- Prosody extraction (CPU - librosa YIN needs CPU) ----
def extract_prosody_generic(path, sr, hop_length, frame_len):
    y, sr = librosa.load(path, sr=sr, mono=True)
    y, _ = librosa.effects.trim(y, top_db=30)
    dur_s = len(y) / sr if sr else np.nan

    f0 = librosa.yin(
        y, fmin=F0_FMIN, fmax=F0_FMAX, sr=sr,
        frame_length=frame_len, hop_length=hop_length
    )
    rms = librosa.feature.rms(
        y=y, frame_length=frame_len, hop_length=hop_length
    ).ravel()

    voiced_mask = np.isfinite(f0) & (f0 > 0) & (rms > (0.5 * np.median(rms)))

    if np.any(voiced_mask):
        idx = np.where(voiced_mask)[0]
        f0_interp = np.interp(np.arange(f0.size), idx, f0[idx])
    else:
        f0_interp = np.full_like(f0, np.nan, dtype=float)

    t = np.arange(f0.size) * (hop_length / sr)
    f0_stats_v = _safe_stats(f0[voiced_mask])
    rms_stats = _safe_stats(rms)

    jitter = (np.median(np.abs(np.diff(f0[voiced_mask]))) / np.median(f0[voiced_mask])
              if np.sum(voiced_mask) > 1 and np.median(f0[voiced_mask]) > 0 else np.nan)
    shimmer = (np.median(np.abs(np.diff(rms[voiced_mask]))) / np.median(rms[voiced_mask])
               if np.sum(voiced_mask) > 1 and np.median(rms[voiced_mask]) > 0 else np.nan)

    slope = (np.polyfit(t[np.isfinite(f0_interp)],
                        f0_interp[np.isfinite(f0_interp)], 1)[0]
             if np.sum(np.isfinite(f0_interp)) >= 2 else np.nan)

    segs = np.diff(np.concatenate([[0], voiced_mask.astype(int), [0]]))
    starts = np.where(segs == 1)[0]
    ends = np.where(segs == -1)[0]
    voiced_durs = (ends - starts) * (hop_length / sr)
    voice_pct = (np.sum(voiced_durs) / dur_s) if dur_s else np.nan

    return dict(
        dur_s=dur_s,
        f0_mean_v=f0_stats_v['mean'], f0_std_v=f0_stats_v['std'],
        f0_iqr_v=f0_stats_v['iqr'], f0_cv_v=f0_stats_v['cv'],
        f0_range_v=f0_stats_v['rng'], f0_p10_v=f0_stats_v['p10'],
        f0_p90_v=f0_stats_v['p90'],
        rms_mean=rms_stats['mean'], rms_std=rms_stats['std'],
        rms_cv=rms_stats['cv'], rms_iqr=rms_stats['iqr'],
        rms_range=rms_stats['rng'],
        jitter_local=jitter, shimmer_local=shimmer,
        f0_slope_hz_per_s=slope,
        voice_pct=voice_pct,
        n_voiced_seg_per_s=(len(voiced_durs) / dur_s if dur_s else np.nan),
        mean_voiced_seg_ms=(np.mean(voiced_durs) * 1000 if len(voiced_durs) else np.nan),
        pause_ratio=(1.0 - voice_pct if voice_pct == voice_pct else np.nan),
        hnr_mean=np.nan
    )

# =========================================================
# GPU-ACCELERATED spectral extraction using torchaudio + MPS
# =========================================================
def extract_spectral_gpu(path, sr, hop_length, n_fft):
    """Extract spectral features using torchaudio on Apple M4 GPU (MPS)."""
    # Load audio with torchaudio
    waveform, orig_sr = torchaudio.load(path)

    # Resample on GPU if needed
    if orig_sr != sr:
        resampler = T.Resample(orig_sr, sr).to(DEVICE)
        waveform = waveform.to(DEVICE)
        waveform = resampler(waveform)
    else:
        waveform = waveform.to(DEVICE)

    # Mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Trim silence using energy threshold on GPU
    energy = waveform.abs().squeeze()
    threshold = energy.max() * (10 ** (-25 / 20))  # top_db=25
    above_thresh = torch.where(energy > threshold)[0]
    if len(above_thresh) > 0:
        waveform = waveform[:, above_thresh[0]:above_thresh[-1]+1]

    # GPU-accelerated spectrogram (STFT on MPS)
    spectrogram_transform = T.Spectrogram(
        n_fft=n_fft, hop_length=hop_length,
        power=2.0, normalized=False
    ).to(DEVICE)

    spec = spectrogram_transform(waveform)  # [1, freq_bins, time_frames]

    # Compute spectral features on GPU
    freqs = torch.linspace(0, sr / 2, spec.shape[1], device=DEVICE)

    # Spectral centroid: weighted mean of frequencies
    spec_sum = spec.squeeze().sum(dim=0) + 1e-10
    centroid = (freqs.unsqueeze(1) * spec.squeeze()).sum(dim=0) / spec_sum

    # Spectral bandwidth: weighted std of frequencies
    deviation = (freqs.unsqueeze(1) - centroid.unsqueeze(0)) ** 2
    bandwidth = torch.sqrt((deviation * spec.squeeze()).sum(dim=0) / spec_sum)

    # Spectral rolloff (85%)
    cumsum = torch.cumsum(spec.squeeze(), dim=0)
    total_energy = cumsum[-1, :] + 1e-10
    rolloff_threshold = 0.85 * total_energy
    rolloff_bins = (cumsum >= rolloff_threshold.unsqueeze(0)).int().argmax(dim=0)
    rolloff = freqs[rolloff_bins]

    # Spectral contrast (simplified: max - min per band)
    n_bands = min(6, spec.shape[1] // 2)
    band_size = spec.shape[1] // n_bands
    contrast_vals = []
    for b in range(n_bands):
        band = spec.squeeze()[b*band_size:(b+1)*band_size, :]
        contrast_vals.append((band.max(dim=0).values - band.min(dim=0).values).mean())
    contrast_mean = torch.stack(contrast_vals).mean()
    contrast_std = torch.stack(contrast_vals).std()

    # Move results to CPU for stats
    centroid_np = centroid.cpu().numpy()
    bandwidth_np = bandwidth.cpu().numpy()
    rolloff_np = rolloff.cpu().numpy()

    def _s(x):
        x = np.asarray(x, float)
        return dict(mean=float(np.mean(x)), std=float(np.std(x)), rng=float(np.ptp(x)))

    cs, bs, rs = _s(centroid_np), _s(bandwidth_np), _s(rolloff_np)

    return dict(
        spec_centroid_mean=cs['mean'], spec_centroid_std=cs['std'],
        spec_centroid_rng=cs['rng'],
        spec_bandwidth_mean=bs['mean'], spec_bandwidth_std=bs['std'],
        spec_bandwidth_rng=bs['rng'],
        spec_contrast_mean=float(contrast_mean.cpu()),
        spec_contrast_std=float(contrast_std.cpu()),
        spec_rolloff_mean=rs['mean'], spec_rolloff_std=rs['std'],
        spec_rolloff_rng=rs['rng'],
    )

# ---- Build feature table ----
def build_feature_table(paths, labels, groups, sr, hop_length, frame_len, tag):
    def _one_p(p, y, g):
        try:
            f = extract_prosody_generic(p, sr=sr, hop_length=hop_length, frame_len=frame_len)
        except Exception:
            f = {k: np.nan for k in [
                'dur_s','f0_mean_v','f0_std_v','f0_iqr_v','f0_cv_v','f0_range_v',
                'f0_p10_v','f0_p90_v','rms_mean','rms_std','rms_cv','rms_iqr',
                'rms_range','jitter_local','shimmer_local','f0_slope_hz_per_s',
                'voice_pct','n_voiced_seg_per_s','mean_voiced_seg_ms',
                'pause_ratio','hnr_mean'
            ]}
        f.update(dict(path=p, label=int(y), group=g))
        return f

    def _one_s_gpu(p, y, g):
        """GPU-accelerated spectral feature extraction."""
        try:
            f = extract_spectral_gpu(p, sr=sr, hop_length=hop_length, n_fft=frame_len)
        except Exception:
            f = {k: np.nan for k in [
                'spec_centroid_mean','spec_centroid_std','spec_centroid_rng',
                'spec_bandwidth_mean','spec_bandwidth_std','spec_bandwidth_rng',
                'spec_contrast_mean','spec_contrast_std',
                'spec_rolloff_mean','spec_rolloff_std','spec_rolloff_rng'
            ]}
        f.update(dict(path=p, label=int(y), group=g))
        return f

    # Prosody: parallel on all CPU cores (YIN is CPU-only)
    print(f'[{tag}] Extracting prosody features (CPU parallel, all cores)...')
    t0 = time.time()
    prosody_rows = Parallel(n_jobs=-1, prefer='threads')(
        delayed(_one_p)(p, y, g) for p, y, g in zip(paths, labels, groups)
    )
    df_prosody = pd.DataFrame(prosody_rows)
    df_prosody = df_prosody[['label', 'group', 'path'] +
                            [c for c in df_prosody.columns
                             if c not in ['label', 'group', 'path']]]
    print(f'  Prosody done in {time.time()-t0:.1f}s')

    # Spectral: GPU-accelerated via torchaudio on MPS
    print(f'[{tag}] Extracting spectral features (GPU MPS accelerated)...')
    t0 = time.time()
    spec_rows = []
    for i, (p, y, g) in enumerate(zip(paths, labels, groups)):
        spec_rows.append(_one_s_gpu(p, y, g))
        if (i + 1) % 5000 == 0:
            print(f'  ... {i+1}/{len(paths)} spectral features extracted')
    df_spec = pd.DataFrame(spec_rows)
    df_spec = df_spec[['label', 'group', 'path'] +
                      [c for c in df_spec.columns
                       if c not in ['label', 'group', 'path']]]
    print(f'  Spectral (GPU) done in {time.time()-t0:.1f}s')

    df_all = df_prosody.merge(df_spec, on=['path', 'label', 'group'], how='outer')
    df_all['sr_tag'] = tag

    print(f'[{tag}] Prosody shape: {df_prosody.shape}'
          f' | Spectral shape: {df_spec.shape}'
          f' | Combined: {df_all.shape}')
    return df_all

# --- Extract at BOTH sample rates ---
print('=== Feature extraction at 44.1 kHz (25ms window, 10ms hop) ===')
print(f'Using device: {DEVICE} for spectral features')
df_all_44k = build_feature_table(
    paths, labels, groups,
    sr=SR_44100, hop_length=HOP_44K, frame_len=FRAME_44K,
    tag='44k_25ms'
)

print('\n=== Feature extraction at 16 kHz (full-clip features) ===')
df_all_16k = build_feature_table(
    paths, labels, groups,
    sr=TARGET_SR_16K, hop_length=HOP_16K, frame_len=FRAME_16K,
    tag='16k_fullclip'
)

# Show sample
print('\n--- Sample of 44.1kHz feature table ---')
print(df_all_44k.head())
print(f'\nShape: {df_all_44k.shape}')
print('\n--- Sample of 16kHz feature table ---')
print(df_all_16k.head())
print(f'\nShape: {df_all_16k.shape}')

=== Feature extraction at 44.1 kHz (25ms window, 10ms hop) ===
Using device: mps for spectral features
[44k_25ms] Extracting prosody features (CPU parallel, all cores)...


  Prosody done in 109.5s
[44k_25ms] Extracting spectral features (GPU MPS accelerated)...
  ... 5000/31138 spectral features extracted


  ... 10000/31138 spectral features extracted
  ... 15000/31138 spectral features extracted


  ... 20000/31138 spectral features extracted


  ... 25000/31138 spectral features extracted
  ... 30000/31138 spectral features extracted


  Spectral (GPU) done in 1.2s
[44k_25ms] Prosody shape: (31138, 24) | Spectral shape: (31138, 14) | Combined: (31138, 36)

=== Feature extraction at 16 kHz (full-clip features) ===
[16k_fullclip] Extracting prosody features (CPU parallel, all cores)...


  Prosody done in 96.7s
[16k_fullclip] Extracting spectral features (GPU MPS accelerated)...
  ... 5000/31138 spectral features extracted


  ... 10000/31138 spectral features extracted
  ... 15000/31138 spectral features extracted


  ... 20000/31138 spectral features extracted
  ... 25000/31138 spectral features extracted


  ... 30000/31138 spectral features extracted
  Spectral (GPU) done in 1.2s
[16k_fullclip] Prosody shape: (31138, 24) | Spectral shape: (31138, 14) | Combined: (31138, 36)

--- Sample of 44.1kHz feature table ---
   label                      group  \
0      1   for-2seconds:file100.wav   
1      1  for-2seconds:file1000.wav   
2      1  for-2seconds:file1001.wav   
3      1  for-2seconds:file1004.wav   
4      1  for-2seconds:file1005.wav   

                                                path  dur_s   f0_mean_v  \
0  /Users/faheemahmad/Downloads/FoR_root/for-2sec...    2.0  175.158454   
1  /Users/faheemahmad/Downloads/FoR_root/for-2sec...    2.0  166.648128   
2  /Users/faheemahmad/Downloads/FoR_root/for-2sec...    2.0  181.722854   
3  /Users/faheemahmad/Downloads/FoR_root/for-2sec...    2.0  163.803925   
4  /Users/faheemahmad/Downloads/FoR_root/for-2sec...    2.0  139.108427   

    f0_std_v   f0_iqr_v   f0_cv_v  f0_range_v    f0_p10_v  ...  \
0  33.073217  43.605222  0.188819  

In [6]:
# =========================================
# 4) EDA: Visualizations & ANOVA
# =========================================

def run_visuals(df_all, tag):
    print(f'\n=== Visualizations for {tag} ===')
    n_per_class = 3000
    df_viz = (df_all.groupby('label', group_keys=False)
                    .apply(lambda d: d.sample(min(len(d), n_per_class),
                                              random_state=SEED))
                    .reset_index(drop=True))

    def split_by_label(df, col):
        a = df.loc[df['label'] == 0, col].replace([np.inf, -np.inf], np.nan).dropna().to_numpy()
        b = df.loc[df['label'] == 1, col].replace([np.inf, -np.inf], np.nan).dropna().to_numpy()
        return a, b

    # F0 distribution
    real, fake = split_by_label(df_viz, 'f0_mean_v')
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.hist(real, bins=50, alpha=0.6, label='real (0)')
    ax.hist(fake, bins=50, alpha=0.6, label='fake (1)')
    ax.set_title(f'Distribution: f0_mean_v ({tag})')
    ax.set_xlabel('f0_mean_v (Hz)')
    ax.set_ylabel('Count')
    ax.legend()
    fig.savefig(os.path.expanduser(f'~/Downloads/f0_dist_{tag}.png'), bbox_inches='tight')
    plt.show()

    # Spectral centroid distribution
    real, fake = split_by_label(df_viz, 'spec_centroid_mean')
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.hist(real, bins=50, alpha=0.6, label='real (0)')
    ax.hist(fake, bins=50, alpha=0.6, label='fake (1)')
    ax.set_title(f'Distribution: spectral centroid mean ({tag})')
    ax.set_xlabel('spec_centroid_mean (Hz)')
    ax.set_ylabel('Count')
    ax.legend()
    fig.savefig(os.path.expanduser(f'~/Downloads/centroid_dist_{tag}.png'), bbox_inches='tight')
    plt.show()

    # Scatter: centroid vs bandwidth
    x0 = df_viz.loc[df_viz['label'] == 0, 'spec_centroid_mean'].to_numpy()
    y0 = df_viz.loc[df_viz['label'] == 0, 'spec_bandwidth_mean'].to_numpy()
    x1 = df_viz.loc[df_viz['label'] == 1, 'spec_centroid_mean'].to_numpy()
    y1 = df_viz.loc[df_viz['label'] == 1, 'spec_bandwidth_mean'].to_numpy()

    fig, ax = plt.subplots(figsize=(7, 5))
    ax.scatter(x0, y0, s=10, alpha=0.5, marker='o', label='real (0)')
    ax.scatter(x1, y1, s=10, alpha=0.5, marker='x', label='fake (1)')
    ax.set_title(f'Spectral Centroid vs Bandwidth ({tag})')
    ax.set_xlabel('spec_centroid_mean')
    ax.set_ylabel('spec_bandwidth_mean')
    ax.legend()
    fig.savefig(os.path.expanduser(f'~/Downloads/scatter_{tag}.png'), bbox_inches='tight')
    plt.show()

def run_anova_and_missing(df_all, tag):
    print(f'\n=== ANOVA + Missing-value summary for {tag} ===')
    exclude = ['label', 'path', 'group', 'sr_tag']
    predictor_cols = [c for c in df_all.columns
                      if c not in exclude and np.issubdtype(df_all[c].dtype, np.number)]

    anova_results = []
    for col in predictor_cols:
        real_vals = df_all[df_all['label'] == 0][col].replace([np.inf, -np.inf], np.nan).dropna()
        fake_vals = df_all[df_all['label'] == 1][col].replace([np.inf, -np.inf], np.nan).dropna()
        if len(real_vals) > 0 and len(fake_vals) > 0:
            F, p = stats.f_oneway(real_vals, fake_vals)
            anova_results.append((col, F, p))

    df_anova = pd.DataFrame(anova_results, columns=['feature', 'F_statistic', 'p_value'])
    df_anova = df_anova.sort_values('p_value')
    print('\nTop 10 most significant features (smallest p):')
    print(df_anova.head(10))
    print('\nBottom 10 least significant features (largest p):')
    print(df_anova.tail(10))

    null_abs = df_all.isna().sum()
    null_pct = (df_all.isna().mean() * 100).round(2)
    df_null_summary = pd.DataFrame({
        'missing_count': null_abs, 'missing_percent': null_pct
    }).sort_values('missing_percent', ascending=False)

    total_missing = df_all.isna().sum().sum()
    total_cells = df_all.size
    overall_pct = round((total_missing / total_cells) * 100, 2)
    print(f'\nOverall missing values ({tag}): '
          f'{total_missing:,} of {total_cells:,} cells ({overall_pct}% missing)\n')
    print(df_null_summary)

    return df_anova, df_null_summary

# Run for both sample rates
run_visuals(df_all_44k, '44k_25ms')
anova_44k, null_44k = run_anova_and_missing(df_all_44k, '44k_25ms')

run_visuals(df_all_16k, '16k_fullclip')
anova_16k, null_16k = run_anova_and_missing(df_all_16k, '16k_fullclip')


=== Visualizations for 44k_25ms ===



=== ANOVA + Missing-value summary for 44k_25ms ===

Top 10 most significant features (smallest p):
          feature  F_statistic        p_value
9         rms_std  1540.662372   0.000000e+00
2        f0_std_v  5237.854095   0.000000e+00
3        f0_iqr_v  1682.366878   0.000000e+00
4         f0_cv_v  5130.223076   0.000000e+00
5      f0_range_v  3502.242116   0.000000e+00
6        f0_p10_v  1994.017893   0.000000e+00
8        rms_mean  2081.873560   0.000000e+00
11        rms_iqr  1957.391791   0.000000e+00
14  shimmer_local   468.813751   0.000000e+00
12      rms_range  1028.015115  5.997354e-222

Bottom 10 least significant features (largest p):
               feature  F_statistic        p_value
10              rms_cv   641.489322  4.168805e-140
7             f0_p90_v   625.141183  1.272546e-136
18  mean_voiced_seg_ms   536.422913  1.120360e-117
16           voice_pct   494.157046  1.245431e-108
19         pause_ratio   494.157046  1.245431e-108
13        jitter_local   331.160563  


=== ANOVA + Missing-value summary for 16k_fullclip ===

Top 10 most significant features (smallest p):
       feature  F_statistic        p_value
2     f0_std_v  4931.438067   0.000000e+00
3     f0_iqr_v  1681.157598   0.000000e+00
4      f0_cv_v  4224.231672   0.000000e+00
5   f0_range_v  3981.260401   0.000000e+00
8     rms_mean  2181.676416   0.000000e+00
9      rms_std  1176.938123  3.290414e-253
10      rms_cv  1160.128541  1.092384e-249
7     f0_p90_v  1106.917513  1.573798e-238
11     rms_iqr  1097.296968  1.646012e-236
12   rms_range  1084.169065  9.400655e-234

Bottom 10 least significant features (largest p):
               feature  F_statistic        p_value
17  n_voiced_seg_per_s   927.004669  1.164632e-200
6             f0_p10_v   836.718092  1.423252e-181
18  mean_voiced_seg_ms   727.214320  2.392369e-158
15   f0_slope_hz_per_s   175.689587   5.433386e-40
0                dur_s   147.413964   7.599739e-34
1            f0_mean_v    42.365594   7.685995e-11
13        jitte

In [7]:
# =========================================
# 5) Model Building & Evaluation
#    Uses GPU (MPS) for SVM via torch where
#    beneficial; sklearn for classical models
# =========================================

def compute_eer_and_curve(y_true, scores):
    fpr, tpr, thresholds = roc_curve(y_true, scores)
    fnr = 1.0 - tpr
    abs_diffs = np.abs(fpr - fnr)
    idx = np.argmin(abs_diffs)
    eer = (fpr[idx] + fnr[idx]) / 2.0
    thr = thresholds[idx]
    return eer, thr, fpr, fnr

def far_frr_at_threshold(y_true, scores, thr):
    y_pred = (scores >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    far = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    frr = fn / (fn + tp) if (fn + tp) > 0 else 0.0
    return far, frr

def run_models(df_all, anova_df, tag):
    print(f'\n================ MODELLING for {tag} ================')

    # Feature selection
    sig_level = 0.05
    weak_features_stat = anova_df.loc[anova_df['p_value'] >= sig_level, 'feature'].tolist()
    all_nan_cols = [c for c in df_all.columns if df_all[c].isna().all()]
    base_drop_cols = ['label', 'path', 'group', 'sr_tag']
    drop_cols = list(set(base_drop_cols + weak_features_stat + all_nan_cols))

    X = df_all.drop(columns=[c for c in drop_cols if c in df_all.columns]) \
             .select_dtypes(include=[np.number]).copy()
    y = df_all['label'].astype(int).to_numpy()
    X.replace([np.inf, -np.inf], np.nan, inplace=True)

    all_nan_cols2 = X.columns[X.isna().all()].tolist()
    if all_nan_cols2:
        X.drop(columns=all_nan_cols2, inplace=True)
        all_nan_cols = list(set(all_nan_cols + all_nan_cols2))

    used_features = X.columns.tolist()

    reason_map = {}
    for f in weak_features_stat:
        reason_map[f] = 'ANOVA p >= 0.05'
    for f in all_nan_cols:
        reason_map[f] = '100% missing'

    dropped_features = [c for c in df_all.columns
                        if c not in used_features + ['label', 'path', 'group', 'sr_tag']]

    print(f'\nFeatures USED [{len(used_features)}]: {used_features}')
    print(f'Features DROPPED [{len(dropped_features)}]:')
    for f in dropped_features:
        print(f'  - {f}: {reason_map.get(f, "Not numeric")}')

    # Train/test split (group-aware)
    if 'group' in df_all.columns:
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
        tr_idx, te_idx = next(gss.split(X, y, groups=df_all['group'].to_numpy()))
    else:
        tr_idx, te_idx = train_test_split(
            np.arange(len(y)), test_size=0.2, stratify=y, random_state=SEED
        )

    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()
    X_train_imp = imputer.fit_transform(X_train)
    X_test_imp = imputer.transform(X_test)
    X_train_sc = scaler.fit_transform(X_train_imp)
    X_test_sc = scaler.transform(X_test_imp)

    print(f'\nTrain shape: {X_train_sc.shape} | Test shape: {X_test_sc.shape}')

    # Models
    model_dict = {
        'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED),
        'LDA': LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'),
        'QDA': QuadraticDiscriminantAnalysis(reg_param=1e-3),
        'GaussianNB': GaussianNB(),
        'SVM_linear': SVC(kernel='linear', C=1.0, class_weight='balanced', random_state=SEED, probability=True),
    }

    # RBF SVM with GridSearchCV
    base_rbf = SVC(kernel='rbf', class_weight='balanced', random_state=SEED, probability=True)
    param_grid = {'C': [0.1, 1, 10], 'gamma': ['scale', 0.01, 0.001]}
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    grid = GridSearchCV(base_rbf, param_grid, cv=skf, scoring='roc_auc', n_jobs=-1)
    print('\nTuning RBF SVM (CV, ROC-AUC)...')
    grid.fit(X_train_sc, y_train)
    rbf_best = grid.best_estimator_
    print('Best RBF params:', grid.best_params_)
    model_dict['SVM_rbf'] = rbf_best

    # GMM (generative baseline)
    gmm_real = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED)
    gmm_fake = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED + 1)
    gmm_real.fit(X_train_sc[y_train == 0])
    gmm_fake.fit(X_train_sc[y_train == 1])

    results_rows = []

    def evaluate_model(name, clf):
        clf.fit(X_train_sc, y_train)
        if hasattr(clf, 'predict_proba'):
            scores_train = clf.predict_proba(X_train_sc)[:, 1]
            scores_test = clf.predict_proba(X_test_sc)[:, 1]
        else:
            scores_train = clf.decision_function(X_train_sc)
            scores_test = clf.decision_function(X_test_sc)

        eer_tr, thr_tr, _, _ = compute_eer_and_curve(y_train, scores_train)
        far_tr, frr_tr = far_frr_at_threshold(y_train, scores_train, thr_tr)
        y_pred_tr = (scores_train >= thr_tr).astype(int)
        acc_tr = accuracy_score(y_train, y_pred_tr)
        roc_tr = roc_auc_score(y_train, scores_train)

        eer_te, thr_te, _, _ = compute_eer_and_curve(y_test, scores_test)
        far_te, frr_te = far_frr_at_threshold(y_test, scores_test, thr_te)
        y_pred_te = (scores_test >= thr_te).astype(int)
        acc_te = accuracy_score(y_test, y_pred_te)
        roc_te = roc_auc_score(y_test, scores_test)

        print(f'\n== {name} ({tag}) ==')
        print(f'[TRAIN] Acc: {acc_tr:.4f} | AUC: {roc_tr:.4f} | EER: {eer_tr*100:.2f}%')
        print(f'[TEST]  Acc: {acc_te:.4f} | AUC: {roc_te:.4f} | EER: {eer_te*100:.2f}%')
        print(f'  FAR@EER: {far_te:.4f} | FRR@EER: {frr_te:.4f}')
        print(f'Classification report (TEST):\n{classification_report(y_test, y_pred_te, digits=4, zero_division=0)}')

        results_rows.append({
            'model': name,
            'accuracy_train': acc_tr, 'roc_auc_train': roc_tr, 'eer_train': eer_tr,
            'far_at_eer_train': far_tr, 'frr_at_eer_train': frr_tr,
            'accuracy': acc_te, 'roc_auc': roc_te, 'eer': eer_te,
            'far_at_eer': far_te, 'frr_at_eer': frr_te
        })

    for name, clf in model_dict.items():
        evaluate_model(name, clf)

    # GMM evaluation
    ll_real_train = gmm_real.score_samples(X_train_sc)
    ll_fake_train = gmm_fake.score_samples(X_train_sc)
    gmm_scores_train = ll_fake_train - ll_real_train

    ll_real_test = gmm_real.score_samples(X_test_sc)
    ll_fake_test = gmm_fake.score_samples(X_test_sc)
    gmm_scores_test = ll_fake_test - ll_real_test

    eer_tr, thr_tr, _, _ = compute_eer_and_curve(y_train, gmm_scores_train)
    far_tr, frr_tr = far_frr_at_threshold(y_train, gmm_scores_train, thr_tr)
    y_pred_tr = (gmm_scores_train >= thr_tr).astype(int)
    acc_tr = accuracy_score(y_train, y_pred_tr)
    roc_tr = roc_auc_score(y_train, gmm_scores_train)

    eer_te, thr_te, _, _ = compute_eer_and_curve(y_test, gmm_scores_test)
    far_te, frr_te = far_frr_at_threshold(y_test, gmm_scores_test, thr_te)
    y_pred_te = (gmm_scores_test >= thr_te).astype(int)
    acc_te = accuracy_score(y_test, y_pred_te)
    roc_te = roc_auc_score(y_test, gmm_scores_test)

    print(f'\n== GMM_logL_diff ({tag}) ==')
    print(f'[TRAIN] Acc: {acc_tr:.4f} | AUC: {roc_tr:.4f} | EER: {eer_tr*100:.2f}%')
    print(f'[TEST]  Acc: {acc_te:.4f} | AUC: {roc_te:.4f} | EER: {eer_te*100:.2f}%')
    print(f'Classification report (TEST):\n{classification_report(y_test, y_pred_te, digits=4, zero_division=0)}')

    results_rows.append({
        'model': 'GMM_logL_diff',
        'accuracy_train': acc_tr, 'roc_auc_train': roc_tr, 'eer_train': eer_tr,
        'far_at_eer_train': far_tr, 'frr_at_eer_train': frr_tr,
        'accuracy': acc_te, 'roc_auc': roc_te, 'eer': eer_te,
        'far_at_eer': far_te, 'frr_at_eer': frr_te
    })

    results_df = pd.DataFrame(results_rows).sort_values('roc_auc', ascending=False)
    print(f'\n=== Summary metrics (sorted by TEST ROC-AUC) for {tag} ===')
    print(results_df)

    # FAR vs FRR plot for best model
    best_name = results_df.iloc[0]['model']
    if best_name == 'GMM_logL_diff':
        scores_best = gmm_scores_test
    else:
        best_clf = model_dict[best_name]
        if hasattr(best_clf, 'predict_proba'):
            scores_best = best_clf.predict_proba(X_test_sc)[:, 1]
        else:
            scores_best = best_clf.decision_function(X_test_sc)

    _, _, fpr_best, fnr_best = compute_eer_and_curve(y_test, scores_best)
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.plot(fpr_best, fnr_best, label=f'{best_name} ({tag})')
    ax.plot([0, 1], [1, 0], '--', linewidth=1)
    ax.set_xlabel('FAR')
    ax.set_ylabel('FRR')
    ax.set_title(f'FAR vs FRR ({best_name}, {tag})')
    ax.legend()
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(os.path.expanduser(f'~/Downloads/far_frr_{tag}.png'), bbox_inches='tight')
    plt.show()

    return {
        'tag': tag,
        'metrics': results_df,
        'features_used': used_features,
        'model_dict': model_dict,
        'gmm_real': gmm_real,
        'gmm_fake': gmm_fake,
        'X_test_sc': X_test_sc,
        'y_test': y_test,
    }

# Run models for both conditions
res_44k = run_models(df_all_44k, anova_44k, '44k_25ms')
res_16k = run_models(df_all_16k, anova_16k, '16k_fullclip')


================ MODELLING for 44k_25ms ================

Features USED [20]: ['dur_s', 'f0_mean_v', 'f0_std_v', 'f0_iqr_v', 'f0_cv_v', 'f0_range_v', 'f0_p10_v', 'f0_p90_v', 'rms_mean', 'rms_std', 'rms_cv', 'rms_iqr', 'rms_range', 'jitter_local', 'shimmer_local', 'f0_slope_hz_per_s', 'voice_pct', 'n_voiced_seg_per_s', 'mean_voiced_seg_ms', 'pause_ratio']
Features DROPPED [12]:
  - hnr_mean: 100% missing
  - spec_centroid_mean: 100% missing
  - spec_centroid_std: 100% missing
  - spec_centroid_rng: 100% missing
  - spec_bandwidth_mean: 100% missing
  - spec_bandwidth_std: 100% missing
  - spec_bandwidth_rng: 100% missing
  - spec_contrast_mean: 100% missing
  - spec_contrast_std: 100% missing
  - spec_rolloff_mean: 100% missing
  - spec_rolloff_std: 100% missing
  - spec_rolloff_rng: 100% missing

Train shape: (24913, 20) | Test shape: (6225, 20)

Tuning RBF SVM (CV, ROC-AUC)...


Exception ignored in: <function ResourceTracker.__del__ at 0x107f71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Best RBF params: {'C': 10, 'gamma': 'scale'}



== LogisticRegression (44k_25ms) ==
[TRAIN] Acc: 0.7117 | AUC: 0.7830 | EER: 28.83%
[TEST]  Acc: 0.7102 | AUC: 0.7806 | EER: 28.98%
  FAR@EER: 0.2901 | FRR@EER: 0.2895
Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7145    0.7099    0.7122      3144
           1     0.7059    0.7105    0.7082      3081

    accuracy                         0.7102      6225
   macro avg     0.7102    0.7102    0.7102      6225
weighted avg     0.7102    0.7102    0.7102      6225


== LDA (44k_25ms) ==
[TRAIN] Acc: 0.7077 | AUC: 0.7728 | EER: 29.23%
[TEST]  Acc: 0.7028 | AUC: 0.7699 | EER: 29.72%
  FAR@EER: 0.2974 | FRR@EER: 0.2970
Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7071    0.7026    0.7049      3144
           1     0.6985    0.7030    0.7007      3081

    accuracy                         0.7028      6225
   macro avg     0.7028    0.7028    0.7028      6225
weighted avg     0


== SVM_linear (44k_25ms) ==
[TRAIN] Acc: 0.7166 | AUC: 0.7815 | EER: 28.34%
[TEST]  Acc: 0.7141 | AUC: 0.7770 | EER: 28.59%
  FAR@EER: 0.2859 | FRR@EER: 0.2859
Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7182    0.7141    0.7161      3144
           1     0.7099    0.7141    0.7120      3081

    accuracy                         0.7141      6225
   macro avg     0.7140    0.7141    0.7140      6225
weighted avg     0.7141    0.7141    0.7141      6225



...

Exception ignored in: <function ResourceTracker.__del__ at 0x106f71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x102dddbc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x1121c9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x107171bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes



== SVM_rbf (16k_fullclip) ==
[TRAIN] Acc: 0.8651 | AUC: 0.9360 | EER: 13.49%
[TEST]  Acc: 0.8432 | AUC: 0.9217 | EER: 15.68%
  FAR@EER: 0.1568 | FRR@EER: 0.1568
Classification report (TEST):
              precision    recall  f1-score   support

           0     0.8459    0.8432    0.8445      3144
           1     0.8405    0.8432    0.8419      3081

    accuracy                         0.8432      6225
   macro avg     0.8432    0.8432    0.8432      6225
weighted avg     0.8432    0.8432    0.8432      6225


== GMM_logL_diff (16k_fullclip) ==
[TRAIN] Acc: 0.7443 | AUC: 0.8175 | EER: 25.57%
[TEST]  Acc: 0.7311 | AUC: 0.8074 | EER: 26.89%
Classification report (TEST):
              precision    recall  f1-score   support

           0     0.7350    0.7312    0.7331      3144
           1     0.7272    0.7309    0.7290      3081

    accuracy                         0.7311      6225
   macro avg     0.7311    0.7311    0.7311      6225
weighted avg     0.7311    0.7311    0.7311    

In [8]:
# =========================================
# 6) McNemar's Pairwise Statistical Tests
# =========================================

def run_mcnemar_all(df_all, anova_df, tag):
    print(f'\n================ McNemar tests for {tag} ================')

    sig_level = 0.05
    weak_features_stat = anova_df.loc[anova_df['p_value'] >= sig_level, 'feature'].tolist()
    all_nan_cols = [c for c in df_all.columns if df_all[c].isna().all()]
    drop_cols = list(set(['label','path','group','sr_tag'] + weak_features_stat + all_nan_cols))

    X = df_all.drop(columns=[c for c in drop_cols if c in df_all.columns]) \
             .select_dtypes(include=[np.number]).copy()
    y = df_all['label'].astype(int).to_numpy()
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    all_nan_cols2 = X.columns[X.isna().all()].tolist()
    if all_nan_cols2:
        X.drop(columns=all_nan_cols2, inplace=True)

    if 'group' in df_all.columns:
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
        tr_idx, te_idx = next(gss.split(X, y, groups=df_all['group'].to_numpy()))
    else:
        tr_idx, te_idx = train_test_split(
            np.arange(len(y)), test_size=0.2, stratify=y, random_state=SEED
        )

    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()
    X_train_sc = scaler.fit_transform(imputer.fit_transform(X_train))
    X_test_sc = scaler.transform(imputer.transform(X_test))

    model_dict = {
        'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED),
        'LDA': LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'),
        'QDA': QuadraticDiscriminantAnalysis(reg_param=1e-3),
        'GaussianNB': GaussianNB(),
        'SVM_linear': SVC(kernel='linear', C=1.0, class_weight='balanced', random_state=SEED, probability=True),
    }

    grid = GridSearchCV(
        SVC(kernel='rbf', class_weight='balanced', random_state=SEED, probability=True),
        {'C': [0.1, 1, 10], 'gamma': ['scale', 0.01, 0.001]},
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED),
        scoring='roc_auc', n_jobs=-1
    )
    grid.fit(X_train_sc, y_train)
    model_dict['SVM_rbf'] = grid.best_estimator_

    gmm_real = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED)
    gmm_fake = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED + 1)
    gmm_real.fit(X_train_sc[y_train == 0])
    gmm_fake.fit(X_train_sc[y_train == 1])

    preds_train, preds_test = {}, {}

    for name, clf in model_dict.items():
        clf.fit(X_train_sc, y_train)
        if hasattr(clf, 'predict_proba'):
            s_tr = clf.predict_proba(X_train_sc)[:, 1]
            s_te = clf.predict_proba(X_test_sc)[:, 1]
        else:
            s_tr = clf.decision_function(X_train_sc)
            s_te = clf.decision_function(X_test_sc)
        _, thr_tr, _, _ = compute_eer_and_curve(y_train, s_tr)
        _, thr_te, _, _ = compute_eer_and_curve(y_test, s_te)
        preds_train[name] = (s_tr >= thr_tr).astype(int)
        preds_test[name] = (s_te >= thr_te).astype(int)

    # GMM
    s_tr_gmm = gmm_fake.score_samples(X_train_sc) - gmm_real.score_samples(X_train_sc)
    s_te_gmm = gmm_fake.score_samples(X_test_sc) - gmm_real.score_samples(X_test_sc)
    _, thr_tr_gmm, _, _ = compute_eer_and_curve(y_train, s_tr_gmm)
    _, thr_te_gmm, _, _ = compute_eer_and_curve(y_test, s_te_gmm)
    preds_train['GMM_logL_diff'] = (s_tr_gmm >= thr_tr_gmm).astype(int)
    preds_test['GMM_logL_diff'] = (s_te_gmm >= thr_te_gmm).astype(int)

    def mcnemar_for_predictions(y_true, preds_dict, which_split):
        model_names = sorted(preds_dict.keys())
        rows = []
        print(f'\n=== McNemar ({which_split}) for {tag} ===')
        for m1, m2 in combinations(model_names, 2):
            correct1 = (preds_dict[m1] == y_true)
            correct2 = (preds_dict[m2] == y_true)
            b = np.sum((correct1 == 0) & (correct2 == 1))
            c = np.sum((correct1 == 1) & (correct2 == 0))
            if b + c == 0:
                chi2_stat, p_value = 0.0, 1.0
            else:
                chi2_stat = (abs(b - c) - 1)**2 / (b + c)
                p_value = chi2.sf(chi2_stat, df=1)
            signif = 'Significant' if p_value < 0.05 else 'Not significant'
            print(f'{m1} vs {m2} | b={b}, c={c} | chi2={chi2_stat:.4f} | p={p_value:.4g} -> {signif}')
            rows.append({'model_1': m1, 'model_2': m2, 'b': int(b), 'c': int(c),
                         'chi2': chi2_stat, 'p_value': p_value, 'significant': p_value < 0.05})
        return pd.DataFrame(rows)

    df_mcn_train = mcnemar_for_predictions(y_train, preds_train, 'TRAIN')
    df_mcn_test = mcnemar_for_predictions(y_test, preds_test, 'TEST')
    return df_mcn_train, df_mcn_test

mcn_train_44k, mcn_test_44k = run_mcnemar_all(df_all_44k, anova_44k, '44k_25ms')
mcn_train_16k, mcn_test_16k = run_mcnemar_all(df_all_16k, anova_16k, '16k_fullclip')


================ McNemar tests for 44k_25ms ================


Exception ignored in: <function ResourceTracker.__del__ at 0x110f31bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x1121f5bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x109d09bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x109d35bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


...

Exception ignored in: <function ResourceTracker.__del__ at 0x105871bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes



=== McNemar (TRAIN) for 44k_25ms ===
GMM_logL_diff vs GaussianNB | b=3875, c=4575 | chi2=57.8226 | p=2.869e-14 -> Significant
GMM_logL_diff vs LDA | b=3735, c=4160 | chi2=22.7709 | p=1.825e-06 -> Significant
GMM_logL_diff vs LogisticRegression | b=3710, c=4034 | chi2=13.4722 | p=0.0002421 -> Significant
GMM_logL_diff vs QDA | b=3654, c=3629 | chi2=0.0791 | p=0.7785 -> Not significant
GMM_logL_diff vs SVM_linear | b=3579, c=3782 | chi2=5.5433 | p=0.01855 -> Significant
GMM_logL_diff vs SVM_rbf | b=5207, c=1374 | chi2=2231.3059 | p=0 -> Significant
GaussianNB vs LDA | b=2196, c=1921 | chi2=18.2356 | p=1.952e-05 -> Significant
GaussianNB vs LogisticRegression | b=2310, c=1934 | chi2=33.1350 | p=8.598e-09 -> Significant
GaussianNB vs QDA | b=2219, c=1494 | chi2=141.1732 | p=1.475e-32 -> Significant
GaussianNB vs SVM_linear | b=2333, c=1836 | chi2=59.0108 | p=1.568e-14 -> Significant
GaussianNB vs SVM_rbf | b=5638, c=1105 | chi2=3045.9772 | p=0 -> Significant
LDA vs LogisticRegression | b=

Exception ignored in: <function ResourceTracker.__del__ at 0x110a71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x1069a9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes



=== McNemar (TRAIN) for 16k_fullclip ===
GMM_logL_diff vs GaussianNB | b=2228, c=3448 | chi2=261.7972 | p=6.963e-59 -> Significant
GMM_logL_diff vs LDA | b=2006, c=3055 | chi2=217.0132 | p=4.054e-49 -> Significant
GMM_logL_diff vs LogisticRegression | b=1980, c=2980 | chi2=201.2099 | p=1.137e-45 -> Significant
GMM_logL_diff vs QDA | b=1690, c=2298 | chi2=92.3894 | p=7.119e-22 -> Significant
GMM_logL_diff vs SVM_linear | b=1786, c=2696 | chi2=184.3554 | p=5.426e-42 -> Significant
GMM_logL_diff vs SVM_rbf | b=4318, c=1308 | chi2=1609.3283 | p=0 -> Significant
GaussianNB vs LDA | b=1833, c=1662 | chi2=8.2690 | p=0.004033 -> Significant
GaussianNB vs LogisticRegression | b=1875, c=1655 | chi2=13.5867 | p=0.0002278 -> Significant
GaussianNB vs QDA | b=2000, c=1388 | chi2=110.1892 | p=8.907e-26 -> Significant
GaussianNB vs SVM_linear | b=1907, c=1597 | chi2=27.2491 | p=1.789e-07 -> Significant
GaussianNB vs SVM_rbf | b=5508, c=1278 | chi2=2635.4909 | p=0 -> Significant
LDA vs LogisticRegres

In [9]:
# =========================================
# 7) Train vs Test Accuracy Bar Charts
# =========================================

def plot_train_test_accuracy(metrics_df, tag):
    order = ['SVM_rbf', 'GMM_logL_diff', 'QDA', 'SVM_linear',
             'LogisticRegression', 'LDA', 'GaussianNB']
    metrics_df = metrics_df.set_index('model').loc[
        [m for m in order if m in metrics_df['model'].values]
    ].reset_index()

    pretty_names = {
        'LogisticRegression': 'Logistic Regression', 'LDA': 'LDA',
        'QDA': 'QDA', 'GaussianNB': 'Gaussian NB',
        'SVM_linear': 'Linear SVM', 'SVM_rbf': 'RBF SVM',
        'GMM_logL_diff': 'GMM',
    }

    model_names = [pretty_names.get(m, m) for m in metrics_df['model']]
    train_accs = metrics_df['accuracy_train'].to_numpy()
    test_accs = metrics_df['accuracy'].to_numpy()

    x = np.arange(len(model_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(x - width/2, train_accs, width, label='Train Accuracy')
    ax.bar(x + width/2, test_accs, width, label='Test Accuracy')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=45)
    ax.set_ylabel('Accuracy')
    ax.set_ylim(0.5, 1.0)
    ax.set_title(f'Train vs Test Accuracy ({tag})')
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.expanduser(f'~/Downloads/accuracy_bar_{tag}.png'), bbox_inches='tight')
    plt.show()

plot_train_test_accuracy(res_16k['metrics'].copy(), '16k_fullclip')
plot_train_test_accuracy(res_44k['metrics'].copy(), '44k_25ms')

In [10]:
# =========================================
# 8) ROC Curves for All Models
# =========================================

def prepare_features_for_roc(df_all, anova_df, tag, sig_level=0.05):
    weak_features = anova_df.loc[anova_df['p_value'] >= sig_level, 'feature'].tolist()
    all_nan_cols = [c for c in df_all.columns if df_all[c].isna().all()]
    drop_cols = list(set(['label','path','group','sr_tag'] + weak_features + all_nan_cols))
    X = df_all.drop(columns=[c for c in drop_cols if c in df_all.columns]) \
              .select_dtypes(include=[np.number]).copy()
    y = df_all['label'].astype(int).to_numpy()
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    nan2 = X.columns[X.isna().all()].tolist()
    if nan2: X.drop(columns=nan2, inplace=True)

    if 'group' in df_all.columns:
        gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
        tr_idx, te_idx = next(gss.split(X, y, groups=df_all['group'].to_numpy()))
    else:
        tr_idx, te_idx = train_test_split(np.arange(len(y)), test_size=0.2, stratify=y, random_state=SEED)

    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    imp = SimpleImputer(strategy='median')
    sc = StandardScaler()
    X_train_sc = sc.fit_transform(imp.fit_transform(X_train))
    X_test_sc = sc.transform(imp.transform(X_test))
    return X_train_sc, X_test_sc, y_train, y_test

def fit_all_models_get_scores(X_train_sc, X_test_sc, y_train, y_test):
    model_dict = {
        'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED),
        'LDA': LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'),
        'QDA': QuadraticDiscriminantAnalysis(reg_param=1e-3),
        'GaussianNB': GaussianNB(),
        'SVM_linear': SVC(kernel='linear', C=1.0, class_weight='balanced', random_state=SEED, probability=True),
    }
    grid = GridSearchCV(
        SVC(kernel='rbf', class_weight='balanced', random_state=SEED, probability=True),
        {'C': [0.1, 1, 10], 'gamma': ['scale', 0.01, 0.001]},
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED),
        scoring='roc_auc', n_jobs=-1
    )
    print('Tuning RBF SVM for ROC...')
    grid.fit(X_train_sc, y_train)
    model_dict['SVM_rbf'] = grid.best_estimator_

    scores_dict = {}
    for name, clf in model_dict.items():
        clf.fit(X_train_sc, y_train)
        if hasattr(clf, 'predict_proba'):
            scores_dict[name] = clf.predict_proba(X_test_sc)[:, 1]
        else:
            scores_dict[name] = clf.decision_function(X_test_sc)

    gmm_real = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED)
    gmm_fake = GaussianMixture(n_components=3, covariance_type='full', random_state=SEED + 1)
    gmm_real.fit(X_train_sc[y_train == 0])
    gmm_fake.fit(X_train_sc[y_train == 1])
    scores_dict['GMM_logL_diff'] = gmm_fake.score_samples(X_test_sc) - gmm_real.score_samples(X_test_sc)
    return scores_dict

def plot_roc_all_models(y_test, scores_dict, title_suffix):
    fig, ax = plt.subplots(figsize=(7, 6))
    for name, scores in scores_dict.items():
        fpr, tpr, _ = roc_curve(y_test, scores)
        auc_val = roc_auc_score(y_test, scores)
        ax.plot(fpr, tpr, label=f'{name} (AUC = {auc_val:.2f})')
    ax.plot([0, 1], [0, 1], 'k--', label='Baseline (AUC = 0.50)')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curves - {title_suffix}')
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.legend(fontsize=8, loc='lower right')
    fig.tight_layout()
    safe_name = title_suffix.replace(' ', '_').replace(',', '')
    fig.savefig(os.path.expanduser(f'~/Downloads/roc_{safe_name}.png'), bbox_inches='tight')
    plt.show()

# 44.1 kHz ROC
Xtr_44, Xte_44, ytr_44, yte_44 = prepare_features_for_roc(df_all_44k, anova_44k, '44k_25ms')
scores_44 = fit_all_models_get_scores(Xtr_44, Xte_44, ytr_44, yte_44)
plot_roc_all_models(yte_44, scores_44, 'FoR 44.1 kHz 25ms window')

# 16 kHz ROC
Xtr_16, Xte_16, ytr_16, yte_16 = prepare_features_for_roc(df_all_16k, anova_16k, '16k_fullclip')
scores_16 = fit_all_models_get_scores(Xtr_16, Xte_16, ytr_16, yte_16)
plot_roc_all_models(yte_16, scores_16, 'FoR 16 kHz full clip')

Tuning RBF SVM for ROC...


Exception ignored in: <function ResourceTracker.__del__ at 0x106471bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Tuning RBF SVM for ROC...


Exception ignored in: <function ResourceTracker.__del__ at 0x107b71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x108375bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


...

Exception ignored in: <function ResourceTracker.__del__ at 0x104431bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x103371bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x105b71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x104495bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x107871bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Exception ignored in: <function ResourceTracker.__del__ at 0x104d71bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


In [11]:
# =========================================
# 9) DET Curves
# =========================================

def plot_det_all_models(y_test, scores_dict, title_suffix):
    fig, ax = plt.subplots(figsize=(7, 6))
    ticks = np.array([0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5])
    tick_labels = [f'{int(t * 100)}%' for t in ticks]

    for name, scores in scores_dict.items():
        fpr_det, fnr_det, _ = det_curve(y_test, scores)
        eps = 1e-6
        fpr_det = np.clip(fpr_det, ticks[0] + eps, 0.5 - eps)
        fnr_det = np.clip(fnr_det, ticks[0] + eps, 0.5 - eps)
        ax.plot(fpr_det, fnr_det, label=name)

    xx = np.linspace(ticks[0], 0.5, 400)
    yy = np.clip(1.0 - xx, ticks[0], 0.5)
    ax.plot(xx, yy, 'k--', label='Baseline')

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_yticks(ticks)
    ax.set_yticklabels(tick_labels)
    ax.set_xlim([ticks[0], 0.5])
    ax.set_ylim([ticks[0], 0.5])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('False Negative Rate')
    ax.set_title(f'DET Curves - {title_suffix}')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend(fontsize=8, loc='lower left')
    fig.tight_layout()
    safe_name = title_suffix.replace(' ', '_').replace(',', '')
    fig.savefig(os.path.expanduser(f'~/Downloads/det_{safe_name}.png'), bbox_inches='tight')
    plt.show()

plot_det_all_models(yte_44, scores_44, 'FoR 44.1 kHz 25ms window')
plot_det_all_models(yte_16, scores_16, 'FoR 16 kHz full clip')

In [12]:
# =========================================
# 10) Correlation Heatmaps (Real vs Fake)
# =========================================

def plot_corr_heatmaps_real_fake(df_all, feature_list, tag):
    cols = [c for c in feature_list
            if c in df_all.columns and np.issubdtype(df_all[c].dtype, np.number)]
    if not cols:
        print(f'No numeric features to plot for {tag}.')
        return

    df_real = df_all[df_all['label'] == 0][cols].dropna(axis=1, how='all')
    df_fake = df_all[df_all['label'] == 1][cols].dropna(axis=1, how='all')

    common_cols = sorted(set(df_real.columns).intersection(set(df_fake.columns)))
    if not common_cols:
        print(f'No common features for {tag}.')
        return

    corr_real = df_real[common_cols].corr()
    corr_fake = df_fake[common_cols].corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    fig.suptitle(f'Feature Correlation Heatmaps (Real vs Fake) - {tag}', fontsize=14)

    for ax, corr, title in zip(axes, [corr_real, corr_fake], ['REAL', 'FAKE']):
        im = ax.imshow(corr.values, vmin=-1, vmax=1, cmap='coolwarm')
        ax.set_title(title)
        ax.set_xticks(np.arange(len(corr.columns)))
        ax.set_yticks(np.arange(len(corr.columns)))
        ax.set_xticklabels(corr.columns, rotation=90)
        ax.set_yticklabels(corr.columns)
        for i in range(corr.shape[0]):
            for j in range(corr.shape[1]):
                ax.text(j, i, f'{corr.values[i, j]:.2f}',
                        ha='center', va='center', fontsize=6, color='black')

    cbar_ax = fig.add_axes([0.47, 0.15, 0.02, 0.7])
    fig.colorbar(im, cax=cbar_ax)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(os.path.expanduser(f'~/Downloads/corr_heatmap_{tag}.png'), bbox_inches='tight')
    plt.show()

plot_corr_heatmaps_real_fake(df_all_44k, res_44k['features_used'], '44k_25ms')
plot_corr_heatmaps_real_fake(df_all_16k, res_16k['features_used'], '16k_fullclip')

In [13]:
# =========================================
# 11) Side-by-Side Comparison & Class Balance
# =========================================

# Comparison of 44.1k vs 16k
m44 = res_44k['metrics'].set_index('model')
m16 = res_16k['metrics'].set_index('model')
comparison = m16.join(m44, how='outer', lsuffix='_16k', rsuffix='_44k')
print('\n=== SIDE-BY-SIDE COMPARISON (16k vs 44k) ===')
print(comparison)

print('\nFeatures USED (44.1k / 25 ms):')
print(res_44k['features_used'])
print('\nFeatures USED (16k):')
print(res_16k['features_used'])

# Class & speaker balance check
def check_class_and_speaker_balance(df_all, tag):
    print(f'\n===== Class balance for {tag} =====')
    y = df_all['label'].to_numpy().astype(int)
    groups = df_all['group'].to_numpy()

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
    train_idx, test_idx = next(gss.split(np.zeros(len(y)), y, groups=groups))

    y_train, y_test = y[train_idx], y[test_idx]
    groups_train, groups_test = groups[train_idx], groups[test_idx]

    print(f'Unique speakers TRAIN: {len(np.unique(groups_train))}')
    print(f'Unique speakers TEST: {len(np.unique(groups_test))}')

    train_counts = np.bincount(y_train, minlength=2)
    test_counts = np.bincount(y_test, minlength=2)
    print(f'Class counts TRAIN [real, fake]: {train_counts}')
    print(f'Class counts TEST  [real, fake]: {test_counts}')
    print(f'Train % fake: {100 * train_counts[1] / train_counts.sum():.2f}%')
    print(f'Test  % fake: {100 * test_counts[1] / test_counts.sum():.2f}%')

check_class_and_speaker_balance(df_all_16k, '16k_fullclip')
check_class_and_speaker_balance(df_all_44k, '44k_25ms')

print('\n=== ALL DONE! ===')


=== SIDE-BY-SIDE COMPARISON (16k vs 44k) ===
                    accuracy_train_16k  roc_auc_train_16k  eer_train_16k  \
model                                                                      
GMM_logL_diff                 0.744270           0.817544       0.255730   
GaussianNB                    0.695300           0.762349       0.304701   
LDA                           0.702164           0.765703       0.297836   
LogisticRegression            0.704130           0.768584       0.295870   
QDA                           0.719865           0.789904       0.280135   
SVM_linear                    0.707743           0.765938       0.292257   
SVM_rbf                       0.865091           0.936007       0.134910   

                    far_at_eer_train_16k  frr_at_eer_train_16k  accuracy_16k  \
model                                                                          
GMM_logL_diff                   0.255643              0.255816      0.731084   
GaussianNB                   